# American Sign Language (ASL) Hand Alphabet Recognition

# 1. Logostic Regression model

## 1a. Train with GPU (Google Colab)

In [3]:
# Install cuml (GPU)

!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 535 (delta 174), reused 130 (delta 94), pack-reused 269 (from 1)
Receiving objects: 100% (535/535), 172.47 KiB | 6.16 MiB/s, done.
Resolving deltas: 100% (276/276), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.2 MB/s eta 0:00:00
Installing RAPIDS remaining 24.10.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.7/567.7 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 577.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.5/915.5 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Imports
import io
import requests

import pandas as pd
import numpy as np
import cuml, cudf

from cuml.linear_model import LogisticRegression #Logistic Regression

from cuml.preprocessing import LabelEncoder

from cuml.model_selection import train_test_split, GridSearchCV

from sklearn.metrics import classification_report, roc_auc_score, auc, roc_curve, confusion_matrix, accuracy_score

In [83]:
# Load dataset

url = 'https://raw.githubusercontent.com/rrsr28/Sign-Language-Recognition/main/Dataset.csv'
#Raw url: 'https://github.com/rrsr28/Sign-Language-Recognition/blob/main/Dataset.csv'

#data = cudf.read_csv(url)
data = pd.read_csv(url)
print((len(data.columns)))

X = data.iloc[:, :-1]
y = data.iloc[:, -1]


64


In [84]:
# Preprocessing

X = np.array(X)

#y = np.array(y)
y = y.to_numpy()
print(type(y))

original_labels = np.unique(y)

encoder = LabelEncoder()
y = encoder.fit_transform(y) # Convert labels to numerical values

y = y.to_numpy()
print(type(y))

numerical_labels = np.unique(y)

for i in np.arange(len(original_labels)):
  print(f'{numerical_labels[i]}: Original label {original_labels[i]}')

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
0: Original label A
1: Original label B
2: Original label C
3: Original label D
4: Original label E
5: Original label F
6: Original label G
7: Original label H
8: Original label I
9: Original label J
10: Original label K
11: Original label L
12: Original label M
13: Original label N
14: Original label No Hand Detected
15: Original label O
16: Original label P
17: Original label Q
18: Original label R
19: Original label S
20: Original label T
21: Original label U
22: Original label V
23: Original label W
24: Original label X
25: Original label Y
26: Original label Z


In [90]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)



In [86]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6399, 63)
(1599, 63)
(6399,)
(1599,)


In [87]:
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
# Train

# Logistic Regression model
params = {
    'C': [0.1, 0.5, 0.8, 0.9, 1, 1.1, 1.2, 2, 2.5],
    #"solver": ["lbfgs", "sag", "saga"] #--> Not possible with cuml
}

logistic_regression_model = LogisticRegression(max_iter=1000)
grid_search_model = GridSearchCV(logistic_regression_model, verbose=2, scoring='accuracy', param_grid=params)

grid_search_model.fit(X_train, y_train)

print(grid_search_model.best_params_)
print(grid_search_model.best_score_)

best_model = grid_search_model.best_estimator_
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ..............................................C=0.1; total time=   0.2s
[CV] END ..............................................C=0.1; total time=   0.1s
[CV] END ..............................................C=0.1; total time=   0.1s
[CV] END ..............................................C=0.1; total time=   0.1s
[CV] END ..............................................C=0.1; total time=   0.1s
[CV] END ..............................................C=0.5; total time=   0.1s
[CV] END ..............................................C=0.5; total time=   0.1s
[CV] END ..............................................C=0.5; total time=   0.1s
[CV] END ..............................................C=0.5; total time=   0.1s
[CV] END ..............................................C=0.5; total time=   0.1s
[CV] END ..............................................C=0.8; total time=   0.1s
[CV] END ........................................

> best params: {'C': 2.5}
0.8879514024628616